COMP 215 - LAB 2 Records (NEO)
----------------
#### Name: David Lu
#### Date: 22 Jan 2024

This lab exercise is mostly a review of strings, tuples, lists, dictionaries, and functions.

**Building on new concepts from lab 1**:
  * `datetime.date` objects represent a calendar date
  * *list comprehension* provides a compact way to represent map and filter algorithms

**New Python Concepts**:
  * *f-string* simplifies string formatting operations

As usual, the first code cell simply imports all the modules we'll be using...

In [22]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

We'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/)
> using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS)

You should register for your own API key, (but may use the DEMO_KEY to get started).

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [23]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

2024-01-22 <class 'datetime.date'>
Today is: 2024-01-22


### Make a query

Let's get some data from the NEO database...
Here's a query that gets the observation "feed" for today.
(Note: I hard-coded the date below to lock down the data for the lab - ideally used `today()` so the notebook is always up-to-date.)

In [24]:
API_KEY = 'MQ8RytTcgwsCHIorzTANVhiyuM5Ghqn230HOkCSd'  # substitute your API key here

today = '2023-01-09'  #  Future enhancement:  str(datetime.date.today())   # Today's date as a string!
# Use an f-string here to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

response = requests.request("GET", url, headers={}, data={})

data = json.loads(response.text)  # recall json.loads for lab 1

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:5])

13 Near Earth Objects found for 2023-01-09
[{'absolute_magnitude_h': 19.63,
  'close_approach_data': [{'close_approach_date': '2023-01-09',
                           'close_approach_date_full': '2023-Jan-09 13:59',
                           'epoch_date_close_approach': 1673272740000,
                           'miss_distance': {'astronomical': '0.0725378623',
                                             'kilometers': '10851509.694433301',
                                             'lunar': '28.2172284347',
                                             'miles': '6742815.4556135138'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '32387.3479203461',
                                                 'kilometers_per_second': '8.9964855334',
                                                 'miles_per_hour': '20124.2384897284'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 2312.1980468998,
     

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [25]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Exercise 1

In the code cell below, **re-write the accumulator loop above** as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [26]:
# Ex. 1 your code here

hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]
print(f'{len(hazards)} potentially hazardous asteroids identified.')

3 potentially hazardous asteroids identified.


## Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.

We can use this `id` to fetch complete orbital and close approach data for the NEO.

For example, this query fetches the complete data set for the first hazardous asteroid...


In [27]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

print(id)
pprint(data['close_approach_data'][:2])

2226554
[{'close_approach_date': '1900-10-30',
  'close_approach_date_full': '1900-Oct-30 21:15',
  'epoch_date_close_approach': -2182819500000,
  'miss_distance': {'astronomical': '0.4183488933',
                    'kilometers': '62584103.354537271',
                    'lunar': '162.7377194937',
                    'miles': '38887958.5659094998'},
  'orbiting_body': 'Earth',
  'relative_velocity': {'kilometers_per_hour': '30027.9671888418',
                        'kilometers_per_second': '8.3411019969',
                        'miles_per_hour': '18658.2110568667'}},
 {'close_approach_date': '1901-06-26',
  'close_approach_date_full': '1901-Jun-26 20:27',
  'epoch_date_close_approach': -2162172780000,
  'miss_distance': {'astronomical': '0.0308794402',
                    'kilometers': '4619498.480712374',
                    'lunar': '12.0121022378',
                    'miles': '2870423.2526198012'},
  'orbiting_body': 'Earth',
  'relative_velocity': {'kilometers_per_hour': '33428

Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".

## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [31]:
# Ex. 2 your code here

def closest_earth_approach (approach_data, ids = []):

    earth_approaches = [apr for apr in approach_data if apr['orbiting_body'] == 'Earth']

    # Initialise the date of closest approach and the minimum distance
    # reached by the NEO at the point of its closest approach
    # with data from the first approach in the list

    date = earth_approaches[0]['close_approach_date']
    minimum = int(float(earth_approaches[0]['miss_distance']['kilometers']))

    if ids != []: id = ids[0]  # for ex3

    # Loop through the remainder of the list. Compare the miss distance
    # of each approach to the minimum distance found so far.
    # If the miss distance is closer, update both the date and distance
    # of closest approach with the values from the new approach.

    for index, approach in enumerate(earth_approaches[1:]):

        miss_km = int(float(approach['miss_distance']['kilometers']))

        if miss_km < minimum:

            minimum = miss_km
            date = approach['close_approach_date']

            if ids != []: id = ids[index+1]  # this is bad... but I'm not sure what's better

    return (date, minimum) if ids == [] else (date, minimum, id)

# ----------
# Unit tests - I have no idea what I'm doing!

test_data = [0]*5

for i in range(5):

    test_data[i] = {

        'orbiting_body': 'Earth',
        'close_approach_date': f'1800-02-{i+1:02d}',
        'miss_distance': {'kilometers': 1000*(i+1)}

    }

# pprint(test_data)

assert closest_earth_approach(test_data) == ('1800-02-01', 1000)

test_data[0]['orbiting_body'] = 'Venus'  # invalidate approach 1
test_data[-1]['miss_distance']['kilometers'] = 10000  # make sure it's not doing something silly like string comparison
test_data[1]['close_approach_date'] = '1800-02-25'  # change the date up because why not

assert closest_earth_approach(test_data) == ('1800-02-25', 2000)

# Try some actual data
# date, minimum = closest_earth_approach(data['close_approach_data'])
# print(date, type(date), minimum, type(minimum))

# test_ids = [i for i in range(len(data['close_approach_data']))]
# date, minimum, id = closest_earth_approach(data['close_approach_data'], test_ids)
# print(date, type(date), minimum, type(minimum), id, (type(id)))

[{'orbiting_body': 'Earth', 'close_approach_date': '1800-02-01', 'miss_distance': {'kilometers': 1000}}, {'orbiting_body': 'Earth', 'close_approach_date': '1800-02-02', 'miss_distance': {'kilometers': 2000}}, {'orbiting_body': 'Earth', 'close_approach_date': '1800-02-03', 'miss_distance': {'kilometers': 3000}}, {'orbiting_body': 'Earth', 'close_approach_date': '1800-02-04', 'miss_distance': {'kilometers': 4000}}, {'orbiting_body': 'Earth', 'close_approach_date': '1800-02-05', 'miss_distance': {'kilometers': 5000}}]
[{'orbiting_body': 'Earth', 'close_approach_date': '1800-02-25', 'miss_distance': {'kilometers': 2000}}, {'orbiting_body': 'Earth', 'close_approach_date': '1800-02-03', 'miss_distance': {'kilometers': 3000}}, {'orbiting_body': 'Earth', 'close_approach_date': '1800-02-04', 'miss_distance': {'kilometers': 4000}}, {'orbiting_body': 'Earth', 'close_approach_date': '1800-02-05', 'miss_distance': {'kilometers': 10000}}]


## Challenge - Take your skills to the next level...
## Exercise 3

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [32]:
# Ex. 3 (challenge) your code here

# --------
# Source: Alex Martelli's answer on SE (https://stackoverflow.com/a/2364277)
# for the combination of generator expressions with the next() function on lines 60, 68
# --------

# Part I: Fetch the IDs of NEOs which have
# flown past Earth in the last seven days

# end = datetime.date.today()
end = datetime.date(2023, 1, 9)
start = end - datetime.timedelta(days=2)

url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start}&end_date={end}&api_key={API_KEY}'

response = requests.get(url)
result = json.loads(response.text)

# pprint(result['near_earth_objects'])
# print(result['near_earth_objects'].keys())

neos = list(result['near_earth_objects'].values())
neo_ids = [neo['id'] for dates in neos for neo in dates]

# Part II: Form a mixed list of approach data from the
# closest approach of each NEO whose ID was fetched previously

closest_approaches = [0] * len(neo_ids)

for i, id_num in enumerate(neo_ids):

    query = f'https://api.nasa.gov/neo/rest/v1/neo/{id_num}?api_key={API_KEY}'
    response = requests.get(query)
    result = json.loads(response.text)

    apr_data = result['close_approach_data']

    closest_approach_neo = {

        'orbiting_body': 'Earth',
        'close_approach_date': closest_earth_approach(apr_data)[0],
        'miss_distance': {'kilometers': closest_earth_approach(apr_data)[1]}

    }

    closest_approaches[i] = closest_approach_neo

# Part III: Compare the closest approach of each NEO and determine
# the NEO which misses Earth by the shortest distance

date, minimum, id = closest_earth_approach(closest_approaches, neo_ids)

# print(f'Approach: {minimum} km on {date}, id = {id}')

t_query = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
t_response = requests.get(t_query)
t_result = json.loads(t_response.text)
t_data = t_result['close_approach_data']

closest_approach = next(apr for apr in t_data if apr['close_approach_date'] == date)
assert int(float(closest_approach['miss_distance']['kilometers'])) == minimum

# pprint(closest_approach)

# Part IV: Print a small blurb about the NEO which makes
# the closest approach to Earth

closest_neo = next(neo for dates in neos for neo in dates if neo['id'] == id)

# pprint(closest_neo)

name = closest_neo['name'][1:-1]
diameter = sum(closest_neo['estimated_diameter']['meters'].values()) / 2
velocity = int(float(closest_approach['relative_velocity']['kilometers_per_hour']))

print(f'Name: {name}')
print(f'Reference ID: {id}')
print(f'Estimated diameter: {diameter:.1f} m\n')

print(f'NEO {name} makes its closest approach to Earth on {date}')
print(f'at a distance of {minimum} km and with a relative velocity of {velocity} km/hr.')


[{'close_approach_date': '1900-05-30', 'close_approach_date_full': '1900-May-30 12:17', 'epoch_date_close_approach': -2196070980000, 'relative_velocity': {'kilometers_per_second': '12.0784345786', 'kilometers_per_hour': '43482.3644830313', 'miles_per_hour': '27018.2503089146'}, 'miss_distance': {'astronomical': '0.3252405849', 'lunar': '126.5185875261', 'kilometers': '48655298.738594163', 'miles': '30233000.7132909294'}, 'orbiting_body': 'Earth'}, {'close_approach_date': '1901-01-09', 'close_approach_date_full': '1901-Jan-09 01:53', 'epoch_date_close_approach': -2176754820000, 'relative_velocity': {'kilometers_per_second': '12.3203191391', 'kilometers_per_hour': '44353.1489006506', 'miles_per_hour': '27559.3218821848'}, 'miss_distance': {'astronomical': '0.0371158713', 'lunar': '14.4380739357', 'kilometers': '5552455.289674131', 'miles': '3450135.7320837678'}, 'orbiting_body': 'Earth'}, {'close_approach_date': '1901-12-31', 'close_approach_date_full': '1901-Dec-31 05:19', 'epoch_date_c

IndexError: list index out of range